In [1]:
from azureml.core.model import Model
import azureml.core

print("SDK version:", azureml.core.VERSION)

from azureml.core import Workspace
ws = Workspace.from_config()
# Tip: When model_path is set to a directory, you can use the child_paths parameter to include
#      only some of the files from the directory

SDK version: 1.0.85


In [2]:
model = Model.register(model_path = "./deploy_files",
                       model_name = "Gen_Model",
                       description = "Generative Model",
                       workspace = ws)

Registering model Gen_Model


In [3]:
from azureml.core.model import InferenceConfig
from azureml.core.environment import Environment
from azureml.core.conda_dependencies import CondaDependencies

# Create the environment
myenv = Environment(name="myenv")
conda_dep = CondaDependencies()
conda_dep.add_pip_package("azureml-defaults")
conda_dep.add_pip_package("torch")
conda_dep.add_pip_package("torchvision")
conda_dep.add_pip_package("tqdm")

myenv.python.conda_dependencies=conda_dep

inference_config = InferenceConfig(entry_script="./score.py",
                                   environment=myenv)

In [4]:
from azureml.core.webservice import LocalWebservice

deployment_config = LocalWebservice.deploy_configuration()

In [5]:
from azureml.core.model import Model

model = Model(ws, name='Gen_Model')
service = Model.deploy(ws, 'genservice1', [model], inference_config, deployment_config)

Generating Docker build context.
Package creation Succeeded
Logging into Docker registry mlmain2c9169d7.azurecr.io
Logging into Docker registry mlmain2c9169d7.azurecr.io
Building Docker image from Dockerfile...
Step 1/5 : FROM mlmain2c9169d7.azurecr.io/azureml/azureml_e5a5dcda2bf731f7dbecefe9da79ae62
 ---> b6034b1d6cde
Step 2/5 : COPY azureml-app /var/azureml-app
 ---> 68da28f625e1
Step 3/5 : RUN mkdir -p '/var/azureml-app' && echo eyJhY2NvdW50Q29udGV4dCI6eyJzdWJzY3JpcHRpb25JZCI6ImYxYmU2MjkzLTZkMWMtNGU3Ny1iYjA2LWIyMDgzZTlhZWIyMCIsInJlc291cmNlR3JvdXBOYW1lIjoic3luYmlvbGljLW1haW4iLCJhY2NvdW50TmFtZSI6Im1sLW1haW4iLCJ3b3Jrc3BhY2VJZCI6IjNmYTk4NGE2LWU5YTEtNGI0MC05YTYxLTI4Mzc2MzU5NzU5NiJ9LCJtb2RlbHMiOnt9LCJtb2RlbHNJbmZvIjp7fX0= | base64 --decode > /var/azureml-app/model_config_map.json
 ---> Running in a6b8b875ab12
 ---> f24ddcbd814b
Step 4/5 : RUN mv '/var/azureml-app/tmpc6h1n_34.py' /var/azureml-app/main.py
 ---> Running in aae31e575b5b
 ---> a8fa3bc1c558
Step 5/5 : CMD ["runsvdir","/var/runi

In [6]:
service.wait_for_deployment(True)
print(service.state)
print("scoring URI: " + service.scoring_uri)

Checking container health...
Local webservice is running at http://localhost:32791
running
scoring URI: http://localhost:32791/score
